In [1]:
%matplotlib inline

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image, ImageOps
from DL import DLLayer, DLModel
from sklearn.metrics import confusion_matrix

In [2]:
data = pd.read_csv('fer2013.csv')
data.head()

,emotion,pixels,Usage
0,0,70 80 82 72 58 58 60 63 54 58 60 48 89 115 121...,Training
1,0,151 150 147 155 148 133 111 140 170 174 182 15...,Training
2,2,231 212 156 164 174 138 161 173 182 200 106 38...,Training
3,4,24 32 36 30 32 23 19 20 30 41 21 22 32 34 21 1...,Training
4,6,4 0 0 0 0 0 0 0 0 0 0 0 3 15 23 28 48 50 58 84...,Training


In [3]:
emotions_names = {0: 'Angry', 1: 'Disgust', 2: 'Fear', 3: 'Happy', 4: 'Sad', 5: 'Surprise', 6: 'Neutral'}
data['emotion_name'] = data['emotion'].map(emotions_names)
data.head()

,emotion,pixels,Usage,emotion_name
0,0,70 80 82 72 58 58 60 63 54 58 60 48 89 115 121...,Training,Angry
1,0,151 150 147 155 148 133 111 140 170 174 182 15...,Training,Angry
2,2,231 212 156 164 174 138 161 173 182 200 106 38...,Training,Fear
3,4,24 32 36 30 32 23 19 20 30 41 21 22 32 34 21 1...,Training,Sad
4,6,4 0 0 0 0 0 0 0 0 0 0 0 3 15 23 28 48 50 58 84...,Training,Neutral


In [4]:
pixels_values = data.pixels.str.split(" ").tolist()
pixels_values = pd.DataFrame(pixels_values, dtype=np.int8)

C:\Users\User\anaconda3\envs\myenv\lib\site-packages\IPython\core\interactiveshell.py:3441: FutureWarning: Could not cast to int8, falling back to object. This behavior is deprecated. In a future version, when a dtype is passed to 'DataFrame', either all columns will be cast to that dtype, or a TypeError will be raised
  exec(code_obj, self.user_global_ns, self.user_ns)


In [5]:
images = pixels_values.values
images = images.astype(float)

In [6]:
print(images.shape)

(35887, 2304)


In [7]:
image_pixels = images.shape[1]
image_pixels

2304

In [8]:
labels_flat = data["emotion"].values.ravel()
labels_count = np.unique(labels_flat).shape[0]

In [9]:
def dense_to_one_hot(labels_dense, num_classes):
    num_labels = labels_dense.shape[0]
    index_offset = np.arange(num_labels) * num_classes
    labels_one_hot = np.zeros((num_labels, num_classes))
    labels_one_hot.flat[[index_offset + labels_dense.ravel()]] = 1
    return labels_one_hot

In [10]:
labels = dense_to_one_hot(labels_flat, labels_count)
labels = labels.astype(np.uint8)
print(labels)

[[1 0 0 ... 0 0 0]
 [1 0 0 ... 0 0 0]
 [0 0 1 ... 0 0 0]
 ...
 [1 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 1 ... 0 0 0]]


In [11]:
m = 32298
X_train, Y_train = images[:m].T, labels[:m].T
X_test, Y_test = images[m:].T, labels[m:].T
print(X_train.shape, Y_train.shape)

(2304, 32298) (7, 32298)


In [28]:
model = DLModel()
model.add(DLLayer("Hidden 1", 180, (2304,), "sigmoid", learning_rate=0.01))
model.add(DLLayer("Output", 7, (180,), "softmax", learning_rate=0.001))

In [29]:
model.compile("categorical_cross_entropy")

In [30]:
costs = model.train(X_train, Y_train, 1500)

cost after  1 %: 1.863761405992696
cost after  2 %: 1.8502169395677892
cost after  3 %: 1.8416871772063959
cost after  4 %: 1.834961230638954
cost after  5 %: 1.8296067554264859
cost after  6 %: 1.825576743974943
cost after  7 %: 1.8221555949164794
cost after  8 %: 1.81922142168055
cost after  9 %: 1.8166907313116163
cost after  10 %: 1.8144807173223152
cost after  11 %: 1.812498399927493
cost after  12 %: 1.8106833285368142
cost after  13 %: 1.809063262934125
cost after  14 %: 1.8075054511485882
cost after  15 %: 1.8060483982599158
cost after  16 %: 1.804662406774318
cost after  17 %: 1.803358878625765
cost after  18 %: 1.8020952276954687
cost after  19 %: 1.8008846851089249
cost after  20 %: 1.7997447864847156
cost after  21 %: 1.7985652598114499
cost after  22 %: 1.797252475459918
cost after  23 %: 1.796114075234514
cost after  24 %: 1.7949276140791794
cost after  25 %: 1.793665807699119
cost after  26 %: 1.7927447683255595
cost after  27 %: 1.791530984294585
cost after  28 %: 1.790

In [ ]:
model.save_weights('weights3')

In [ ]:
plt.plot(np.squeeze(costs))
plt.ylabel('cost')
plt.xlabel('iterations')
plt.show()

In [ ]:
def predict_softmax(X, Y, model):
    AL = model.predict(X)
    predictions = np.argmax(AL, axis=0)
    labels = np.argmax(Y, axis=0)
    return confusion_matrix(predictions, labels)

In [ ]:
print('Deep train accuracy')
pred_train = predict_softmax(X_train, Y_train, model)
print(pred_train)

In [ ]:
print('Deep test accuracy')
pred_test = predict_softmax(X_test, Y_test, model)
print(pred_test)

In [ ]:
i = 4
print('train', str(i), str(pred_train[i][i] / np.sum(pred_train[:, i])))
print('test', str(i), str(pred_test[i][i] / np.sum(pred_test[:, i])))